In [3]:
import nltk
nltk.download('stopwords')
import re
from tqdm import tqdm_notebook
from nltk.corpus import stopwords
from tensorflow.keras import regularizers, initializers, optimizers, callbacks
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

#!spell login #henmja, a****

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
Using TensorFlow backend.


In [5]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
data = pd.read_pickle("bigdataHashed.pkl")
data.tail()

,contributors_enabled_user,created_at,created_at_user,default_profile_image_user,default_profile_user,description_user,display_url_zero_urls_entities,display_url_zero_urls_url_entities_user,expanded_url_zero_urls_entities,expanded_url_zero_urls_url_entities_user,favorite_count,favorited,favourites_count_user,follow_request_sent_user,followers_count_user,following_user,friends_count_user,geo_enabled_user,has_extended_profile_user,id,id_str,id_str_user,id_str_zero_user_mentions_entities,id_user,id_zero_user_mentions_entities,is_quote_status,is_translation_enabled_user,is_translator_user,lang,listed_count_user,location_user,name_user,name_zero_user_mentions_entities,notifications_user,possibly_sensitive,possibly_sensitive_appealable,profile_background_color_user,profile_background_image_url_https_user,profile_background_image_url_user,profile_background_tile_user,profile_banner_url_user,profile_image_url_https_user,profile_image_url_user,profile_link_color_user,profile_sidebar_border_color_user,profile_sidebar_fill_color_user,profile_text_color_user,profile_use_background_image_user,protected_user,retweet_count,retweeted,screen_name_user,screen_name_zero_user_mentions_entities,source,statuses_count_user,text,translator_type_user,truncated,url_user,url_zero_urls_entities,url_zero_urls_url_entities_user,verified_user,zero_indices_zero_urls_entities,one_indices_zero_urls_entities,zero_indices_zero_urls_url_entities_user,one_indices_zero_urls_url_entities_user,zero_indices_zero_user_mentions_entities,one_indices_zero_user_mentions_entities,followers,following,label
788,False,2017-05-20 14:11:39+00:00,Sun Oct 25 14:01:48 +0000 2009,False,True,,fb.me/1fYsjNaLC,thekind-heartedsmartaleck.blogspot.com,http://fb.me/1fYsjNaLC,http://thekind-heartedsmartaleck.blogspot.com/,0.0,False,16.0,False,47.0,False,87.0,True,True,8.659330e+17,8.659330e+17,85085807,10228272,8.508581e+07,8.529181e+16,False,False,False,en,7.0,Gassaway WV,Terry Bly,YouTube,False,False,False,C0DEED,https://abs.twimg.com/images/themes/theme1/bg.png,http://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/13552184...,https://pbs.twimg.com/profile_images/249393875...,http://pbs.twimg.com/profile_images/2493938757...,1DA1F2,C0DEED,DDEEF6,333333,True,False,0.0,False,TlbBly,YouTube,"<a href=""http://www.facebook.com/twitter"" rel=...",90486.0,WORSE THAN HITLER! Trey Gowdy’s Son Found In A...,none,False,None,https://t.co/dxpndRqHjL,http://t.co/PYrnTHpCvI,False,93.0,116.0,0.0,22.545205,49.264865,60.043243,8165876438847090636,8165876438847090636,fake
789,False,2017-12-09 09:10:03+00:00,Wed Feb 24 22:58:57 +0000 2016,False,True,PROUD PATRIOT SUPPORTING TRUMP!! PRO-AMERICAN....,twitter.com/i/web/status/9…,thekind-heartedsmartaleck.blogspot.com,https://twitter.com/i/web/status/9394218766187...,http://thekind-heartedsmartaleck.blogspot.com/,0.0,False,74641.0,False,6903.0,False,4533.0,True,True,9.394219e+17,9.394219e+17,702628810483765249,10228272,7.026288e+17,8.529181e+16,False,False,False,en,86.0,"Orlando, FL",#TRUMP2020,YouTube,False,False,False,F5F8FA,None,None,False,https://pbs.twimg.com/profile_banners/70262881...,https://pbs.twimg.com/profile_images/111902430...,http://pbs.twimg.com/profile_images/1119024306...,1DA1F2,C0DEED,DDEEF6,333333,True,False,0.0,False,PolyPatriot,YouTube,"<a href=""http://twitter.com/download/iphone"" r...",155849.0,Facebook Picks Sides in Alabama Senate Race=&g...,none,True,None,https://t.co/o4Qq2UFraj,http://t.co/PYrnTHpCvI,False,120.0,143.0,0.0,22.545205,49.264865,60.043243,3430018387555,3430018387555,fake
790,False,2017-08-29 17:54:38+00:00,Tue Sep 15 02:55:59 +0000 2009,False,True,,houstonchronicle-tv.com/floyd-mayweath…,thekind-heartedsmartaleck.blogspot.com,http://houstonchronicle-tv.com/floyd-mayweathe...,http://thekind-heartedsmartaleck.blogspot.com/,1.0,False,11.0,False,24.0,False,42.0,True,False,9.025903e+17,9.025903e+17,74346965,10228272,7.434696e+07,8.529181e+16,False,False,False,en,0.0,Louisville Ky,Eugene 

In [ ]:
#https://towardsdatascience.com/fake-news-classification-using-glove-and-long-short-term-memory-lstm-a48f1dd605ab
#http://digital-thinking.de/deep-learning-combining-numerical-and-text-features-in-deep-neural-networks/
    
df = pd.read_pickle("./finalData.pkl")
print(df.head())
print(df.info())

#https://datascience.stackexchange.com/questions/67145/word-embedding-or-hash
#last ned user_followers med winSCP
#flytt followers til slutten (før rename)
#skriv til finalDataPKL
#last opp github

#set CAT_ATTRS
#word embedding av text og user_description med vanlig keras modell
#merge embedding og resten av attributtene med vanlig keras modell
#LSTM med vanlig keras modell
#word embed followers og following

In [ ]:
df['created_at'] = bigdataPKL['created_at'].astype(str)

In [ ]:

CAT_ATTRS = df['id'] + " " + df['']
x = df['created_at'] + " " + df['created_at_user'] + " " + df['name_user'] + " " + df['name_zero_user_mentions_entities'] + " " + df['location_user'] + " " + df['description_user'] + " " + df['text'] + " " + df['source']
y = pd.get_dummies(df[‘label’])
y = np.array(y)

print(x[0])

print(y)

print(y.head())

MAX_NB_WORDS = 100000 # max number of words for tokenizer
MAX_SEQUENCE_LENGTH = 1000 # max length of each sentences, including padding
VALIDATION_SPLIT = 0.2 # 20% of data for validation (not used in training)
EMBEDDING_DIM = 100 # embedding dimensions for word vectors
GLOVE_DIR = “glove/glove.6B.”+str(EMBEDDING_DIM)+”d.txt”

def clean_text(text, remove_stopwords = True):
    output = “”
    text = str(text).replace(r’http[\w:/\.]+’,’’) # removing urls
    text = str(text).replace(r’[^\.\w\s]’,’’) #remove everything but characters and punctuation
    text = str(text).replace(r’\.\.+’,’.’) #replace multiple periods with a single one
    text = str(text).replace(r’\.’,’ . ‘) #replace periods with a single one
    text = str(text).replace(r’\s\s+’,’ ‘) #replace multiple white space with a single one
    text = str(text).replace(“\n”, “”) #removing line breaks
    text = re.sub(r’[^\w\s]’,’’,text).lower() #lower texts
    if remove_stopwords:
        text = text.split(“ “)
    for word in text:
        if word not in stopwords.words(“english”):
            output = output + “ “ + word
        else:
            output = text
    return str(output.strip())[1:-3].replace(“ “, “ “)

texts = [] 
for line in tqdm_notebook(x, total=df.shape[0]): 
    texts.append(clean_text(line))

print(‘Sample data:’, texts[0], y[0])

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print(‘Vocabulary size:’, len(word_index))

data = pad_sequences(sequences, padding = ‘post’, maxlen = MAX_SEQUENCE_LENGTH)
print(‘Shape of data tensor:’, data.shape)
print(‘Shape of label tensor:’, y.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = y[indices]

num_validation_samples = int(VALIDATION_SPLIT*data.shape[0])
x_train = data[: -num_validation_samples]
y_train = labels[: -num_validation_samples]
x_val = data[-num_validation_samples: ]
y_val = labels[-num_validation_samples: ]
print(‘Number of entries in each category:’)
print(‘training: ‘, y_train.sum(axis=0))
print(‘validation: ‘, y_val.sum(axis=0))

print(‘Tokenized sentences: \n’, data[10])
print(‘One hot label: \n’, labels[10])

embeddings_index = {}
f = open(GLOVE_DIR, encoding=’utf8')
print(‘Loading Glove from:’, GLOVE_DIR,’…’, end=’’)
for line in f:
 values = line.split()
 word = values[0]
 embeddings_index[word] = np.asarray(values[1:], dtype=’float32')
f.close()
print(“Done.\n Proceeding with Embedding Matrix…”, end=””)
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
 embedding_vector = embeddings_index.get(word)
 if embedding_vector is not None:
 embedding_matrix[i] = embedding_vector
print(“Completed!”)

                                     
input1 = Input(shape=(MAX_SEQ_LENGTH,), name='nlp_input') #TEXT
input2 = Input(shape=(10,), name='meta_input') #NON-TEXT ATTRS
emb = Embedding(output_dim=len(word_index) + 1, input_dim=EMBEDDING_DIM, weights = [embedding_matrix], 
                input_length=MAX_SEQ_LENGTH, 
                trainable=False, 
                name = ‘embeddings’)))(input1)
input1_out = Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=regularizers.l2(0.01)))(emb)
merged = keras.layers.Concatenate(axis=1)([input1_out, input2])
maxpool = keras.layers.GlobalMaxPool1D()(merged)
dropout1 = keras.layers.Dropout(0.1)(maxpool)
dense1 = keras.layers.Dense(1, activation=keras.activations.relu, use_bias=True)(dropout1)
dropout2 = keras.layers.Dropout(0.1)(dense1)
dense2 = keras.layers.Dense(2, input_dim=2, activation=keras.activations.sigmoid, use_bias=True)(dropout2)
model10 = keras.models.Model(inputs=[input1, input2], output=output)
model10.compile(optimizer=’adam’, loss=’binary_crossentropy’,
metrics = ['accuracy')
history = model0.fit([x_train, CAT_ATTRS],y_train, batch_size=16, epochs=100, validation_data=(x_val, y_val))                                     
                
import matplotlib.pyplot as plt
%matplotlib inline
loss = history.history[‘loss’]
val_loss = history.history[‘val_loss’]
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, label=’Training loss’)
plt.plot(epochs, val_loss, label=’Validation loss’)
plt.title(‘Training and validation loss’)
plt.xlabel(‘Epochs’)
plt.ylabel(‘Loss’)
plt.legend()
plt.show();
                
accuracy = history.history[‘acc’]
val_accuracy = history.history[‘val_acc’]
plt.plot(epochs, accuracy, label=’Training accuracy’)
plt.plot(epochs, val_accuracy, label=’Validation accuracy’)
plt.title(‘Training and validation accuracy’)
plt.ylabel(‘Accuracy’)
plt.xlabel(‘Epochs’)
plt.legend()
plt.show();
                
random_num = np.random.randint(0, 100)
test_data = x[random_num]
test_label = y[random_num]
clean_test_data = clean_text(test_data)
test_tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
test_tokenizer.fit_on_texts(clean_test_data)
test_sequences = tokenizer.texts_to_sequences(clean_test_data)
word_index = test_tokenizer.word_index
test_data_padded = pad_sequences(test_sequences, padding = ‘post’, maxlen = MAX_SEQUENCE_LENGTH)
                
prediction = model.predict(test_data_padded)
                
prediction[random_num].argsort()[-len(prediction[random_num]):]
#import numpy as np
#import pandas as pd
#from sklearn.preprocessing import MinMaxScaler
#from tensorflow.contrib.rnn import *
#from keras.models import Sequential
#from keras.layers import Dense, CuDNNGRU, Dropout, LSTM, Flatten, CuDNNLSTM
#from keras.optimizers import adam


#class RNN:
#    def __init__(self, num_features: int):
#        self.history = None
#        self.num_features = num_features
#        self.scaler = MinMaxScaler(feature_range=(0, self.num_features))

#        self.model = Sequential()
#        self.model.add(LSTM(64, input_shape=(self.num_features, 1)))
#        self.model.add(Dense(32, activation='relu'))
#        self.model.add(Dense(1))
#        self.model.compile(optimizer=adam(lr=0.005), loss="mae")

#    def train(self, data, batch_size: int = 32, epochs: int = 20):
#        feature_train, target_train = self.generate_feature_and_target(data)
#        self.history = self.model.fit(feature_train, target_train, epochs=epochs, batch_size=batch_size)
#        return self.history

#    def forecast(self, feature: pd.core.frame.DataFrame):
#        feature_test = feature.reshape(-1, self.num_features)
#        scaled = self.scaler.transform(feature_test)

#        feature_test = np.array(scaled[:, :self.num_features])
#        feature_test = np.reshape(feature_test, (feature_test.shape[0], feature_test.shape[1], 1))

#        forecasts = self.model.predict(feature_test)
#        return forecasts[-1]

#    def generate_feature_and_target(self, data: pd.core.frame.DataFrame):
#        feature_train = self.scaler.fit_transform(data[:, :self.num_features])
#        feature_train, target_train = np.array(feature_train[:, :self.num_features]), np.array(data[:, self.num_features])

#        feature_train = np.reshape(feature_train, (feature_train.shape[0], feature_train.shape[1], 1))

#        return feature_train, target_train